# Wikidata query

In [1]:
import pandas as pd
import requests
from src.apiQueries import getLabelFromFBID, getInfoFromTMDB, multiprocess_query



DATA_PATH = './datasets/'
MOVIE_DATASET_PATH = DATA_PATH + 'MovieSummaries/'
char_datapath = MOVIE_DATASET_PATH + 'character.metadata.tsv'
movie_metadata = MOVIE_DATASET_PATH + 'movie.metadata.tsv'
name_clusters = MOVIE_DATASET_PATH + 'name.clusters.txt'
plot_summaries = MOVIE_DATASET_PATH + 'plot_summaries.txt'
tvtropes_clusters = MOVIE_DATASET_PATH + 'tvtropes.clusters.txt'

In [2]:
# ## Define columns names corresponding to characters metadata 
# ## and import the table using Pandas
# char_col_names= ['Wikipedia_movie_ID','Freebase_move_ID', 'movie_release_date',
#                    'Character_name','Actor_DOB','Actor_gender',
#                    'Actor_height','Actor_ethnicity','Actor_name',
#                    'Actor_age_release','Freebase_character_map',
#                    'Freebase_character_map2','Freebase_character_map3']

# char_metadata_df = pd.read_csv(char_datapath,header=None,sep='\t',names=char_col_names) 

In [3]:
# # Extract the list of Freebase ID ethnicities and get their label
# # with a query on the wikiData api
# FBids = char_metadata_df['Actor_ethnicity'].dropna().unique()
# df = getLabelFromFBID(id_list=FBids,verbose=True)

In [4]:
# # Save the labels for later use (if needed)
# df.to_csv('./generated/EthnicitiesLabels.csv')

In [5]:
# ethn_df = char_metadata_df.dropna(subset=['Actor_ethnicity'])
# ethn_df = ethn_df.merge(right = df,
#                        how='left',
#                        left_on='Actor_ethnicity',
#                        right_on='freebaseID.value')

In [6]:
# ethn_df['freebaseID.value'].isna().sum()

We see that some of the freebase IDs were not found on the database -> wanna search on another DB ?

## IMDB Queries

In [7]:
col_names = names=['Wikipedia_movie_ID','Freebase_move_ID',
                   'Movie_name','Movie_release_date','Movie_box_office_revenue',
                   'Movie_runtime','Movie_languages','Movie_countries','Movie_genres']


movie_metadata_df = pd.read_csv(movie_metadata,header=None,sep='\t',
                                names=col_names) 
movie_list = movie_metadata_df[['Movie_name','Movie_runtime']]

In [8]:
import tmdbsimple as tmdb
with open('api_keys/tmdbkey.txt','r') as file:
    key = file.readline()

In [9]:
length_quartile = len(movie_list)//50
length_quartile

1634

In [10]:
getInfoFromTMDB(movie_list=movie_list[:10],key=key,verbose=True)

Starting to fetch ...
Found a match for movie number 0
['Ghosts of Mars', 5.125, 982, 28000000]
Found a match for movie number 2
['Brun bitter', 0.0, 0, 0]
Found a match for movie number 3
['White Of The Eye', 5.742, 64, 0]
Found a match for movie number 4
['A Woman in Flames', 5.3, 13, 0]
Found a match for movie number 5
['The Gangsters', 6.0, 1, 0]
Found a match for movie number 6
["The Sorcerer's Apprentice", 4.6, 11, 0]
Found a match for movie number 7
["Alexander's Ragtime Band", 6.6, 23, 2000000]


,title,vote_average,vote_counts,budget
0,Ghosts of Mars,5.125,982,28000000
1,Brun bitter,0.000,0,0
2,White Of The Eye,5.742,64,0
3,A Woman in Flames,5.300,13,0
4,The Gangsters,6.000,1,0
5,The Sorcerer's Apprentice,4.600,11,0
6,Alexander's Ragtime Band,6.600,23,2000000


In [11]:

def task(chunk, key):
    return getInfoFromTMDB(movie_list = chunk,key = key, verbose=False)
movie_info_df = multiprocess_query(movie_list=movie_list[:10],api_key=key, nb_workers=1,task = task)

In [ ]:
movie_info_df.to_csv('./generated/TMDB1.csv')